# 第 1 阶段：基础 Agent + 工具调用

本 Notebook 用于学习和调试第 1 阶段的核心功能：
- 配置加载与验证
- LLM 模型创建
- 工具调用（时间、计算器、网络搜索等）
- Agent 基本功能（同步/异步、流式输出）

**参考文档**: `docs/stage_01/`  
**参考脚本**: `scripts/test_basic.py`

## 1. 环境准备

In [ ]:
# 添加项目根目录到 Python 路径
import sys
from pathlib import Path

# 获取 backend 目录路径
backend_dir = Path.cwd()
if backend_dir.name == 'notebooks':
    backend_dir = backend_dir.parent

if str(backend_dir) not in sys.path:
    sys.path.insert(0, str(backend_dir))

print(f"✅ 项目根目录: {backend_dir}")

## 2. 配置加载与验证

In [ ]:
from config import settings, get_logger

# 初始化日志
logger = get_logger(__name__)

print("=" * 60)
print("配置信息")
print("=" * 60)
print(f"应用名称: {settings.app_name}")
print(f"版本: {settings.app_version}")
print(f"OpenAI 模型: {settings.openai_model}")
print(f"API Base: {settings.openai_api_base}")
print(f"温度参数: {settings.openai_temperature}")
print(f"日志级别: {settings.log_level}")
print("=" * 60)

In [ ]:
# 验证必需配置
try:
    settings.validate_required_keys()
    print("✅ 配置验证通过！")
except Exception as e:
    print(f"❌ 配置验证失败: {e}")
    print("\n请检查 .env 文件，确保 OPENAI_API_KEY 已设置")

## 3. LLM 模型创建

In [ ]:
from core.models import get_chat_model

# 创建默认模型
model = get_chat_model()

print(f"✅ 模型创建成功")
print(f"模型类型: {model.__class__.__name__}")
print(f"模型名称: {settings.openai_model}")

In [ ]:
# 测试简单调用
response = model.invoke("你好，请用一句话介绍你自己")
print("\n模型响应:")
print(response.content)

## 4. 工具调用测试

### 4.1 时间工具

In [ ]:
from core.tools import get_current_time, get_current_date

# 测试时间工具
time_result = get_current_time.invoke({})
date_result = get_current_date.invoke({})

print(f"⏰ 当前时间: {time_result}")
print(f"📅 当前日期: {date_result}")

### 4.2 计算器工具

In [ ]:
from core.tools import calculator

# 测试计算器
test_expressions = [
    "2 + 2",
    "10 * 5 + 3",
    "(100 - 20) / 4",
    "2 ** 10"  # 2的10次方
]

print("🧮 计算器测试:")
for expr in test_expressions:
    result = calculator.invoke({"expression": expr})
    print(f"  {expr} = {result}")

### 4.3 网络搜索工具（可选）

In [ ]:
from core.tools import web_search_simple

# 检查是否配置了 Tavily API Key
if settings.tavily_api_key:
    print("🔍 测试网络搜索...")
    search_result = web_search_simple.invoke({"query": "LangChain 是什么"})
    print(f"\n搜索结果:\n{search_result}")
else:
    print("⚠️  未配置 TAVILY_API_KEY，跳过网络搜索测试")
    print("   如需使用，请在 .env 中设置 TAVILY_API_KEY")

### 4.4 查看所有可用工具

In [ ]:
from core.tools import ALL_TOOLS, BASIC_TOOLS

print(f"📦 基础工具数量: {len(BASIC_TOOLS)}")
print(f"📦 所有工具数量: {len(ALL_TOOLS)}")

print("\n基础工具列表:")
for tool in BASIC_TOOLS:
    print(f"  - {tool.name}: {tool.description}")

## 5. Agent 基本功能

### 5.1 创建基础 Agent

In [ ]:
from agents import create_base_agent

# 创建不带工具的 Agent
agent_simple = create_base_agent(mode="default")
print("✅ 创建简单 Agent 成功（无工具）")

# 创建带工具的 Agent
agent_with_tools = create_base_agent(mode="default", use_tools=True)
print("✅ 创建 Agent 成功（带工具）")

### 5.2 简单对话测试

In [ ]:
# 测试简单对话
response = agent_simple.invoke("请用一句话解释什么是机器学习")
print("💬 Agent 回答:")
print(response)

### 5.3 工具调用测试

In [ ]:
# 测试时间查询（需要调用工具）
response = agent_with_tools.invoke("现在几点了？")
print("⏰ Agent 回答（调用时间工具）:")
print(response)

In [ ]:
# 测试计算（需要调用计算器）
response = agent_with_tools.invoke("帮我计算 123 * 456 等于多少")
print("🧮 Agent 回答（调用计算器）:")
print(response)

### 5.4 流式输出测试

In [ ]:
# 创建支持流式输出的 Agent
agent_streaming = create_base_agent(mode="default", streaming=True)

print("📡 流式输出测试:")
print("="*60)

# 流式输出
for chunk in agent_streaming.stream("讲一个关于编程的笑话"):
    print(chunk, end="", flush=True)

print("\n" + "="*60)

## 6. 不同模式的 Agent

### 6.1 编程助手模式

In [ ]:
agent_coding = create_base_agent(mode="coding")

response = agent_coding.invoke("如何在 Python 中实现单例模式？")
print("👨‍💻 编程助手回答:")
print(response)

### 6.2 研究助手模式

In [ ]:
agent_research = create_base_agent(mode="research")

response = agent_research.invoke("简述深度学习的发展历程")
print("🔬 研究助手回答:")
print(response)

### 6.3 简洁模式

In [ ]:
agent_concise = create_base_agent(mode="concise")

response = agent_concise.invoke("什么是 Docker？")
print("⚡ 简洁模式回答:")
print(response)

### 6.4 详细模式

In [ ]:
agent_detailed = create_base_agent(mode="detailed")

response = agent_detailed.invoke("什么是 Docker？")
print("📖 详细模式回答:")
print(response)

## 7. 对话历史管理

In [ ]:
# 创建带历史记录的 Agent
agent = create_base_agent(mode="default")

# 第一轮对话
response1 = agent.invoke("我叫小明，我喜欢编程")
print("第一轮:")
print(response1)

# 第二轮对话（测试是否记住上下文）
response2 = agent.invoke("我叫什么名字？我喜欢什么？")
print("\n第二轮（测试记忆）:")
print(response2)

## 8. 总结

通过本 Notebook，你已经学习了：

✅ 配置加载与验证  
✅ LLM 模型创建与调用  
✅ 各种工具的使用（时间、计算器、搜索）  
✅ Agent 的创建与基本功能  
✅ 同步调用与流式输出  
✅ 不同模式的 Agent（编程、研究、简洁、详细）  
✅ 对话历史管理

**下一步**: 学习第 2 阶段 - RAG 知识库（`stage_02_rag.ipynb`）